In [0]:
%run ./rank_builder_2.8

16:13:58 - PipelineRunner - INFO - 🔧 PipelineBuilder initialized (schema: exec_ccmd_ussocom_personnel_restricted_analytics)
16:13:58 - PipelineRunner - INFO - ✅ Added Bronze step: bronze_main
16:13:58 - PipelineRunner - INFO - ✅ Added Silver step: silver_main (source: bronze_main)
16:14:15 - PipelineRunner - INFO - ✅ Added Silver step (validation-only): silver_old
16:14:15 - PipelineRunner - INFO - ✅ Added Gold step (validation-only): gold_old
16:14:15 - PipelineRunner - INFO - 🔍 Auto-inferred source_silvers: ['silver_main', 'silver_old']
16:14:15 - PipelineRunner - INFO - ✅ Added Gold step: gold_main (sources: ['silver_main', 'silver_old'])
16:14:15 - PipelineRunner - INFO - ✅ Pipeline validation passed
16:14:15 - PipelineRunner - INFO - Starting dependency analysis with strategy: hybrid
16:14:15 - PipelineRunner - INFO - Dependency analysis completed in 0.00s
16:14:15 - PipelineRunner - INFO - 📋 Execution order (5 steps): bronze_main → silver_main → silver_old → gold_old → gold_main
16:14:15 - PipelineRunner - INFO - 🚀 Pipeline built successfully with 1 bronze, 2 silver, 2 gold steps

In [0]:
from pprint import pprint
from pyspark.sql import functions as F, DataFrame
import typing as t

### Run Pipeline

In [0]:
# ─────────────────────── Example: Initial build ───────────────────────
bronze_df: DataFrame = (
    spark.table("dmdc_limited_pii.active_duty_master_prod")
    .withColumn("otpt_file_dt", F.to_date(F.col("otpt_file_dt").cast("string"), "yyyyMMdd"))
    .select(list(bronze_rules.keys()))
    .filter(F.col("otpt_file_dt") > F.lit('2024-01-01'))
    .filter(F.col("otpt_file_dt") < F.lit('2024-06-01'))
)
bronze_df

Out[107]: DataFrame[ppln_cd: string, pn_ssn_id_hash: string, pri_svc_occ_cd: string, otpt_file_dt: date, pgrd_id: string, svc_cd: string]

In [0]:
spark.table('exec_ccmd_ussocom_personnel_restricted_analytics.silver_navy_sof_personnel_3')

Out[108]: DataFrame[pn_ssn_id_hash: string, specialty: string, personnel_type: string, rank_code: string, pay_grade: int, service_branch: string, is_sof: boolean, source_date: date, rank_order: int, years_in_service: double, processed_timestamp: timestamp]

In [0]:
bronze_sources: dict[str, DataFrame] = {"bronze_main": bronze_df, "bronze_second": bronze_df}
initial_report = pipe.run_initial_load(bronze_sources=bronze_sources)
pprint(initial_report.to_dict())

16:14:16 - PipelineRunner - INFO - Starting pipeline execution: pipeline_20260129_161416
16:14:16 - PipelineRunner - INFO - Starting dependency analysis with strategy: hybrid
16:14:16 - PipelineRunner - INFO - Dependency analysis completed in 0.00s
16:14:16 - PipelineRunner - INFO - Dependency analysis complete: 5 steps to execute
16:14:16 - PipelineRunner - INFO - ▶️ Starting BRONZE step: bronze_main
16:16:57 - PipelineRunner - INFO - ✅ Completed BRONZE step: bronze_main (161.26s) - 6489060 rows processed, 0 rows written, 0 invalid, 100.0% valid
16:16:57 - PipelineRunner - INFO - ▶️ Starting SILVER step: silver_old
16:18:33 - PipelineRunner - ERROR - ❌ Failed SILVER step: silver_old (95.96s) - The schema of your Delta table has changed in an incompatible way since your DataFrame or
DeltaTable object was created. Please redefine your DataFrame or DeltaTable object.
Changes:
Latest schema is missing field(s): personnel_type, years_in_service, source_date, rank_order, specialty, processed_timestamp, service_branch, pn_ssn_id_hash, is_sof, pay_grade, rank_code
This check can be turned off by setting the session configuration key
spark.databricks.delta.checkLatestSchemaOnRead to false.
16:18:33 - PipelineRunner - ERROR - Exception executing step silver_old: Step execution failed: The schema of your Delta table has changed in an incompatible way since your DataFrame or
DeltaTable object was created. Please redefine your DataFrame or DeltaTable object.
Changes:
Latest schema is missing field(s): personnel_type, years_in_service, source_date, rank_order, specialty, processed_timestamp, service_branch, pn_ssn_id_hash, is_sof, pay_grade, rank_code
This check can be turned off by setting the session configuration key
spark.databricks.delta.checkLatestSchemaOnRead to false.
16:18:33 - PipelineRunner - INFO - ▶️ Starting GOLD step: gold_old
16:18:34 - PipelineRunner - ERROR - ❌ Failed GOLD step: gold_old (0.90s) - Validation-only gold step 'gold_old' requires existing table 'exec_ccmd_ussocom_personnel_restricted_analytics.gold_navy_sof_ranked_personnel_3', but table does not exist
16:18:34 - PipelineRunner - ERROR - Exception executing step gold_old: Step execution failed: Validation-only gold step 'gold_old' requires existing table 'exec_ccmd_ussocom_personnel_restricted_analytics.gold_navy_sof_ranked_personnel_3', but table does not exist
16:18:34 - PipelineRunner - INFO - ▶️ Starting SILVER step: silver_main
16:18:48 - PipelineRunner - INFO - ✅ Completed SILVER step: silver_main (14.28s) - 1617354 rows processed, 1617354 rows written, 0 invalid, 100.0% valid
16:18:49 - PipelineRunner - INFO - ▶️ Starting GOLD step: gold_main
16:19:02 - PipelineRunner - INFO - ✅ Completed GOLD step: gold_main (13.17s) - 335771 rows processed, 335771 rows written, 0 invalid, 100.0% valid
16:19:02 - PipelineRunner - ERROR - Pipeline execution failed: 2 steps failed
16:19:02 - PipelineRunner - INFO - Completed pipeline execution: pipeline_20260129_161416
{'bronze_results': {'bronze_main': {'duration': 161.255943,
 'end_time': '2026-01-29T16:16:57.788729',
 'input_rows': 6489060,
 'output_table': None,
 'rows_processed': 6489060,
 'rows_written': None,
 'start_time': '2026-01-29T16:14:16.532786',
 'status': 'completed',
 'validation_rate': 100.0,
 'write_mode': None}},
 'duration_seconds': 286.087328,
 'end_time': '2026-01-29T16:19:02.254435',
 'errors': ['Step execution failed: The schema of your Delta table has changed '
 'in an incompatible way since your DataFrame or\n'
 'DeltaTable object was created. Please redefine your DataFrame or '
 'DeltaTable object.\n'
 'Changes:\n'
 'Latest schema is missing field(s): personnel_type, '
 'years_in_service, source_date, rank_order, specialty, '
 'processed_timestamp, service_branch, pn_ssn_id_hash, is_sof, '
 'pay_grade, rank_code\n'
 'This check can be turned off by setting the session configuration '
 'key\n'
 'spark.databricks.delta.checkLatestSchemaOnRead to false.',
 "Step execution failed: Validation-only

In [0]:
# ───────────────────── Create Pipeline Logs Table ────────────────────
logs_table_name = "navy_sof_ranked_personnel_logs_2"
log_writer = LogWriter(spark, schema=write_schema, table_name=logs_table_name)
log_writer.create_table(initial_report)


16:19:02 - LogWriter - INFO - LogWriter initialized for table: exec_ccmd_ussocom_personnel_restricted_analytics.navy_sof_ranked_personnel_logs_2
16:19:02 - LogWriter - INFO - Started monitoring create_table operation: b08de61f-b619-4718-9c33-e70618ecf02c
16:19:02 - LogWriter - INFO - 📊 Creating log table exec_ccmd_ussocom_personnel_restricted_analytics.navy_sof_ranked_personnel_logs_2 for run f0fcffcb-1790-4284-8379-d157332edb58
16:19:02 - LogWriter - INFO - Creating table if not exists: exec_ccmd_ussocom_personnel_restricted_analytics.navy_sof_ranked_personnel_logs_2
16:19:03 - LogWriter - INFO - Table exec_ccmd_ussocom_personnel_restricted_analytics.navy_sof_ranked_personnel_logs_2 exists and is a Delta table (verified via DeltaTable)
16:19:04 - LogWriter - INFO - Writing batch of 5 log rows
16:19:04 - LogWriter - INFO - Writing DataFrame to exec_ccmd_ussocom_personnel_restricted_analytics.navy_sof_ranked_personnel_logs_2 with mode overwrite
🔍 write_dataframe: About to write Delta table exec_ccmd_ussocom_personnel_restricted_analytics.navy_sof_ranked_personnel_logs_2
🔍 write_dataframe: PID=25351
🔍 write_dataframe: Session ID (Python)=140387999346944
🔍 write_dataframe: Session ID (JVM)=140387999347232
🔍 write_dataframe: Pre-write config check - Extensions: '', Catalog: ''
⚠️ write_dataframe: WARNING - Delta configs missing before write!
⚠️ write_dataframe: This will likely cause DeltaAnalysisException
🔍 write_dataframe: Executing saveAsTable(exec_ccmd_ussocom_personnel_restricted_analytics.navy_sof_ranked_personnel_logs_2)
✅ write_dataframe: saveAsTable succeeded
16:19:09 - LogWriter - INFO - Successfully wrote 5 rows to exec_ccmd_ussocom_personnel_restricted_analytics.navy_sof_ranked_personnel_logs_2
16:19:09 - LogWriter - INFO - Completed monitoring b08de61f-b619-4718-9c33-e70618ecf02c: 6.92s, 5 rows
16:19:09 - LogWriter - INFO - ✅ Successfully created log table exec_ccmd_ussocom_personnel_restricted_analytics.navy_sof_ranked_personnel_logs_2 with 5 row(s) for run f0fcffcb-1790-4284-8379-d157332edb58
Out[110]: {'success': True,
 'run_id': 'f0fcffcb-1790-4284-8379-d157332edb58',
 'operation_id': 'b08de61f-b619-4718-9c33-e70618ecf02c',
 'rows_written': 5,
 'table_fqn': 'exec_ccmd_ussocom_personnel_restricted_analytics.navy_sof_ranked_personnel_logs_2',
 'write_result': {'table_name': 'exec_ccmd_ussocom_personnel_restricted_analytics.navy_sof_ranked_personnel_logs_2',
 'write_mode': 'overwrite',
 'rows_written': 5,
 'timestamp': '2026-01-29T16:19:09.222710',
 'success': True},
 'operation_metrics': {'operation_id': 'b08de61f-b619-4718-9c33-e70618ecf02c',
 'success': True,
 'duration_secs': 6.922826290130615,
 'rows_written': 5,
 'memory_usage_mb': 6418.66,
 'error_message': None,
 'timestamp': '2026-01-29T16:19:09.781146'}}

In [0]:
# ───────────────────── Example: Incremental append 1000 new rows ────────────────────
bronze_df: DataFrame = (
    spark.table("dmdc_limited_pii.active_duty_master_prod")
    .withColumn("otpt_file_dt", F.to_date(F.col("otpt_file_dt").cast("string"), "yyyyMMdd"))
    .select(list(bronze_rules.keys()))
    .filter(F.col("otpt_file_dt") > F.lit('2024-01-01'))
    .filter(F.col("otpt_file_dt") < F.lit('2024-06-01'))
)

bronze_sources: dict[str, DataFrame] = {"bronze_main": bronze_df, "bronze_second": bronze_df}
incr_report = pipe.run_incremental(bronze_sources=bronze_sources)
pprint(incr_report.to_dict())

16:19:10 - PipelineRunner - INFO - Starting pipeline execution: pipeline_20260129_161910
16:19:10 - PipelineRunner - INFO - Starting dependency analysis with strategy: hybrid
16:19:10 - PipelineRunner - INFO - Dependency analysis completed in 0.00s
16:19:10 - PipelineRunner - INFO - Dependency analysis complete: 5 steps to execute
16:19:10 - PipelineRunner - INFO - ▶️ Starting BRONZE step: bronze_main
16:19:15 - PipelineRunner - INFO - ✅ Completed BRONZE step: bronze_main (4.67s) - 6489060 rows processed, 0 rows written, 0 invalid, 100.0% valid
16:19:15 - PipelineRunner - INFO - ▶️ Starting SILVER step: silver_old
16:19:25 - PipelineRunner - INFO - ✅ Completed SILVER step: silver_old (10.39s) - 3234708 rows processed, 3234708 rows written, 0 invalid, 100.0% valid
16:19:25 - PipelineRunner - INFO - ▶️ Starting GOLD step: gold_old
16:19:26 - PipelineRunner - ERROR - ❌ Failed GOLD step: gold_old (0.87s) - Validation-only gold step 'gold_old' requires existing table 'exec_ccmd_ussocom_personnel_restricted_analytics.gold_navy_sof_ranked_personnel_3', but table does not exist
16:19:26 - PipelineRunner - ERROR - Exception executing step gold_old: Step execution failed: Validation-only gold step 'gold_old' requires existing table 'exec_ccmd_ussocom_personnel_restricted_analytics.gold_navy_sof_ranked_personnel_3', but table does not exist
16:19:26 - PipelineRunner - INFO - ▶️ Starting SILVER step: silver_main
16:19:41 - PipelineRunner - INFO - Silver step silver_main: filtering bronze rows where otpt_file_dt <= 2024-05-31
16:21:31 - PipelineRunner - INFO - ✅ Completed SILVER step: silver_main (124.48s) - 0 rows processed, 0 rows written, 0 invalid, 100.0% valid
16:21:31 - PipelineRunner - INFO - ▶️ Starting GOLD step: gold_main

In [0]:
log_writer.append(incr_report)

In [0]:
bronze_df: DataFrame = (
    spark.table("dmdc_limited_pii.active_duty_master_prod")
    .withColumn("otpt_file_dt", F.to_date(F.col("otpt_file_dt").cast("string"), "yyyyMMdd"))
    .select(list(bronze_rules.keys()))
    .filter(F.col("otpt_file_dt") > F.lit('2025-01-01'))
    .filter(F.col("otpt_file_dt") < F.lit('2025-06-01'))
)

bronze_sources: dict[str, DataFrame] = {"bronze_main": bronze_df, "bronze_second": bronze_df}
incr_report = pipe.run_incremental(bronze_sources=bronze_sources)
pprint(incr_report.to_dict())

In [0]:
log_writer.append(incr_report)

In [0]:
cols = ['start_time', 'phase', 'step_name', 'write_mode', 'valid_rows', 'invalid_rows', 'table_total_rows']
spark.table(f'{write_schema}.{logs_table_name}')[cols].sort(F.col('start_time')).show()

In [0]:
spark.table('exec_ccmd_ussocom_personnel_restricted_analytics.silver_navy_sof_personnel_2').count()